In [1]:
import cv2
import os

# 1. Setup dictionaries for ASCII mapping
d = {}
c = {}
for i in range(256):
    d[chr(i)] = i  # Character to ASCII integer
    c[i] = chr(i)  # ASCII integer to Character

# --- ENCRYPTION/HIDING PROCESS ---
try:
    # 2. Load the image (NOTE: You must change the file path to a valid image on your system)
    image_path = r"C:\Users\reuab\Pictures\Screenshot_20250512_032409.png"  # <-- CHANGE THIS PATH!
    if not os.path.exists(image_path):
        print(f"Error: Image not found at {image_path}. Please update the path.")
        exit()

    x = cv2.imread(image_path)
    
    if x is None:
        print("Error: Could not load the image. Check the path and file format.")
        exit()

    i_rows, j_cols, _ = x.shape
    print(f"Image dimensions: Rows={i_rows}, Cols={j_cols}")

    # 3. Get user input
    key = input("Enter key to edit (Security Key): ")
    text = input("Enter text to hide: ")
    
    # Check if text is too long for the image size
    if len(text) > i_rows * j_cols:
        print("Error: The text is too long for this image size.")
        exit()

    # 4. Hiding logic setup
    kl = 0  # Key length index
    z = 0   # Color channel index (0=B, 1=G, 2=R)
    n = 0   # Row index
    m = 0   # Column index
    l = len(text)

    # 5. Hide the text
    for char_index in range(l):
        # Steganography: XOR the character's ASCII value with the key character's ASCII value
        # and replace the pixel's color component value with the result.
        
        # The logic was flawed because it was modifying the row/column/channel too quickly.
        # It's better to modify the color channel 'z' for a pixel (n, m) and then move to the next pixel.
        
        # Get the XORed value
        xor_value = d[text[char_index]] ^ d[key[kl]]
        
        # Modify the pixel value
        x[n, m, z] = xor_value
        
        # Move to the next channel for the same pixel (n, m)
        z = (z + 1) % 3  # Cycle through 0, 1, 2 (B, G, R)

        # If all 3 channels of a pixel are used, move to the next pixel (column)
        if z == 0:
            m = m + 1
            
            # If the column limit is reached, move to the next row
            if m == j_cols:
                n = n + 1
                m = 0  # Reset column index
                
                # Check if we ran out of rows
                if n == i_rows:
                    print("Warning: Used all available pixels but text fit.")
                    break

        # Move to the next key character, cycling through the key
        kl = (kl + 1) % len(key)

    # 6. Save the new image
    output_filename = "encrypted_img.png" # Use .png to avoid compression artifacts which can destroy the hidden data
    cv2.imwrite(output_filename, x)
    os.startfile(output_filename)
    print(f"\nData Hiding in Image completed successfully. File saved as {output_filename}")

except KeyError as e:
    print(f"\nError: Character '{e.args[0]}' not in the ASCII map (0-255).")
except Exception as e:
    print(f"\nAn unexpected error occurred: {e}")

# --- DECRYPTION/EXTRACTION PROCESS ---

print("\n" + "="*50)
ch = input("Enter '1' to extract data from Image (or any other key to exit): ")

if ch == '1':
    key1 = input("Re enter key to extract text: ")
    decrypt = ""
    
    if key == key1:
        # Reset counters for extraction
        kl = 0
        z = 0
        n = 0
        m = 0
        l = len(text) # Use the length of the originally hidden text

        # Load the newly saved image (optional, but good practice if running separately)
        # x_decrypted = cv2.imread(output_filename) 

        # 7. Extract the text
        for char_index in range(l):
            # Get the modified pixel value
            encrypted_value = x[n, m, z]
            
            # Decrypt: XOR the value with the key character's ASCII value
            decrypted_value = encrypted_value ^ d[key1[kl]]
            
            # Append the decrypted character
            decrypt += c[decrypted_value]
            
            # Follow the exact same movement logic as in hiding
            z = (z + 1) % 3
            if z == 0:
                m = m + 1
                if m == j_cols:
                    n = n + 1
                    m = 0
            
            kl = (kl + 1) % len(key1)

        print("\nDecrypted text is:", decrypt)
    else:
        print("\nKey doesn't match. Extraction failed.")
else:
    print("Thank you. Exiting.")

Image dimensions: Rows=2340, Cols=1080


Enter key to edit (Security Key):  1
Enter text to hide:  2



Data Hiding in Image completed successfully. File saved as encrypted_img.png



Enter '1' to extract data from Image (or any other key to exit):  1
Re enter key to extract text:  1



Decrypted text is: 2
